In [ ]:
import os
import sys
import numpy as np
from PIL import Image
from flask import Flask, request, render_template, send_from_directory

# --- Try to import heavy libraries and handle potential errors ---
try:
    # Import TensorFlow and Keras components
    # Make sure to install tensorflow: pip install tensorflow
    from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
    from tensorflow.keras.models import Model
    from tensorflow.keras.preprocessing import image as keras_image
    from sklearn.metrics.pairwise import cosine_similarity
except ImportError as e:
    print(" FATAL ERROR: A required library is not installed.")
    print(f"   Error details: {e}")
    print("   Please run 'pip install -r requirements.txt' in your terminal.")
    sys.exit(1)


# --- Configuration ---
# Create necessary directories
if not os.path.exists('static/uploads'):
    os.makedirs('static/uploads')
if not os.path.exists('cricketer_images'):
    os.makedirs('cricketer_images')
    print("INFO: 'cricketer_images' directory created. Please add reference images of cricketers here.")
    print("      For example, create a folder 'cricketer_images/Virat_Kohli/' and add images of him inside.")


# --- Flask App Initialization ---
app = Flask(__name__)
app.config['UPLOAD_FOLDER'] = 'static/uploads'
app.config['CRICKETER_IMAGES_FOLDER'] = 'cricketer_images'

# --- Model Loading ---
def get_model():
    """Load the pre-trained VGG16 model and create a feature extractor."""
    base_model = VGG16(weights='imagenet', include_top=True)
    # We will use the output of the 'fc1' layer as our feature vector
    model = Model(inputs=base_model.input, outputs=base_model.get_layer('fc1').output)
    return model

# --- Feature Extraction ---
def preprocess_image(img_path):
    """Load and preprocess an image for VGG16."""
    img = keras_image.load_img(img_path, target_size=(224, 224))
    img_array = keras_image.img_to_array(img)
    img_array_expanded = np.expand_dims(img_array, axis=0)
    return preprocess_input(img_array_expanded)

def extract_features(model, img_path):
    """Extract deep features from an image using the loaded model."""
    preprocessed_img = preprocess_image(img_path)
    features = model.predict(preprocessed_img, verbose=0)
    return features.flatten()

# --- Pre-compute features for known cricketers (UPDATED LOGIC) ---
def get_cricketer_features(model):
    """
    Extract and store features for all reference cricketer images.
    It now looks for subdirectories inside 'cricketer_images', where each
    subdirectory name is the cricketer's name. It averages the features
    from all images found within each subdirectory.
    """
    cricketer_features_list = {}  # Temp dict to hold lists of features
    final_cricketer_features = {} # Final dict with averaged features
    base_path = app.config['CRICKETER_IMAGES_FOLDER']

    try:
        cricketer_folders = [d for d in os.listdir(base_path) if os.path.isdir(os.path.join(base_path, d))]
    except FileNotFoundError:
        print(f" ERROR: The '{base_path}' directory was not found.")
        return {}

    if not cricketer_folders:
        print(" WARNING: No cricketer subdirectories found in 'cricketer_images' directory.")
        print("           Please create a folder for each cricketer and add their images inside.")
        return {}

    # Iterate through each cricketer's folder
    for folder_name in cricketer_folders:
        cricketer_name = folder_name.replace('_', ' ').title()
        cricketer_features_list[cricketer_name] = []
        folder_path = os.path.join(base_path, folder_name)
        
        image_files = [f for f in os.listdir(folder_path) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]

        if not image_files:
            print(f"   - No images found for {cricketer_name}, skipping.")
            continue

        print(f"Processing reference images for: {cricketer_name}...")
        # Extract features for each image in the folder
        for img_file in image_files:
            img_path = os.path.join(folder_path, img_file)
            try:
                features = extract_features(model, img_path)
                cricketer_features_list[cricketer_name].append(features)
            except Exception as e:
                print(f"   - Could not process image {img_file}. Error: {e}")

    # Average the features for each cricketer
    for name, features_list in cricketer_features_list.items():
        if features_list:  # Ensure the list is not empty
            averaged_features = np.mean(features_list, axis=0)
            final_cricketer_features[name] = averaged_features
    
    if not final_cricketer_features:
        print(" WARNING: Feature extraction completed, but no valid features were generated.")
        return {}

    print("✅ Reference features computed and averaged successfully!")
    return final_cricketer_features

# --- Initialize Model and Features with Error Handling ---
try:
    model = get_model()
    print("✅ Model loaded successfully!")
    known_cricketer_features = get_cricketer_features(model)
except Exception as e:
    print(" FATAL ERROR: Failed to load the model or process reference images.")
    print("   This could be due to an issue with TensorFlow, a corrupted model file, or a problem with the images.")
    print(f"   Error details: {e}")
    sys.exit(1)


# --- Flask Routes ---
@app.route('/', methods=['GET', 'POST'])
def index():
    if request.method == 'POST':
        if 'file' not in request.files:
            return render_template('index.html', error='No file part in the request.')
        
        file = request.files['file']
        
        if file.filename == '':
            return render_template('index.html', error='No file selected.')

        if file and file.filename.lower().endswith(('.png', '.jpg', '.jpeg')):
            filepath = os.path.join(app.config['UPLOAD_FOLDER'], file.filename)
            file.save(filepath)

            uploaded_image_features = extract_features(model, filepath)

            best_match = None
            highest_similarity = -1

            if not known_cricketer_features:
                 return render_template('index.html', 
                                       error="Cannot predict. No reference images loaded.",
                                       image_path=filepath)

            print("\n--- Similarity Scores ---")
            for name, features in known_cricketer_features.items():
                similarity = cosine_similarity(
                    uploaded_image_features.reshape(1, -1),
                    features.reshape(1, -1)
                )[0][0]
                
                print(f"  - Similarity with {name}: {similarity:.4f}")
                
                if similarity > highest_similarity:
                    highest_similarity = similarity
                    best_match = name

            print(f"--- Best Match: {best_match} with similarity {highest_similarity:.4f} ---")
            
            # You can adjust this threshold value based on your testing.
            threshold = 0.6 
            prediction = best_match if highest_similarity > threshold else "Cricketer Not Recognized"
            
            return render_template('index.html',
                                   prediction=prediction,
                                   similarity=f"{highest_similarity*100:.2f}",
                                   image_path=filepath)
        else:
            return render_template('index.html', error='Invalid file type. Please upload a JPG or PNG image.')

    return render_template('index.html')

@app.route('/uploads/<filename>')
def uploaded_file(filename):
    """Serve uploaded files."""
    return send_from_directory(app.config['UPLOAD_FOLDER'], filename)

app.run()



✅ Model loaded successfully!
Processing reference images for: Bhuvneshwar Kumar...
Processing reference images for: Dinesh Karthik...
Processing reference images for: Hardik Pandya...
Processing reference images for: Jasprit Bumrah...
Processing reference images for: K. L. Rahul...
Processing reference images for: Kedar Jadhav...
Processing reference images for: Kuldeep Yadav...
Processing reference images for: Mohammed Shami...
Processing reference images for: Ms Dhoni...
Processing reference images for: Ravindra Jadeja...
Processing reference images for: Rohit Sharma...
Processing reference images for: Shikhar Dhawan...
Processing reference images for: Vijay Shankar...
Processing reference images for: Virat Kohli...
Processing reference images for: Yuzvendra Chahal...
✅ Reference features computed and averaged successfully!
 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
